# **Sentiment Analysis with Deep Learning using BERT**


## **What is BERT?**

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found here (https://arxiv.org/abs/1810.04805).

HuggingFace documentation (https://huggingface.co/transformers/model_doc/bert.html)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1: Exploratory Data Analysis and Preprocessing

In [ ]:
import torch
from tqdm.notebook import tqdm

In [ ]:
df = pd.read_csv('translated_corpus.csv',
                names=['text', 'category'])
df = df[1:]
df.insert(0, 'id', range(1, 1 + len(df)))
df.set_index('id', inplace=True)

In [ ]:
df.head()

,text,category
id,,
1,Tonight peda segura,neutral
2,Eres such mala little woman shrew interesada,negative
3,Yo kiero Pretzels lol,neutral
4,Fuck that nor ke the myself vaya in order to u...,negative
5,I always tell my dad ke myself kiero kasar con...,negative


In [ ]:
df.category.value_counts()

positive    8262
neutral     5669
negative    3063
lang2          4
Name: category, dtype: int64

In [ ]:
df = df[df['category'] != 'lang2']

In [ ]:
df.category.value_counts()

positive    8262
neutral     5669
negative    3063
Name: category, dtype: int64

In [ ]:
possible_labels = df.category.unique()

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
label_dict

{'negative': 1, 'neutral': 0, 'positive': 2}

In [ ]:
df.category = df['category'].map(label_dict)

In [ ]:
df.head(10)

,text,category
id,,
1,Tonight peda segura,0
2,Eres such mala little woman shrew interesada,1
3,Yo kiero Pretzels lol,0
4,Fuck that nor ke the myself vaya in order to u...,1
5,I always tell my dad ke myself kiero kasar con...,1
6,Ke myself compre an shopping cart pa irme con ...,0
7,Why can I just find in order to rich bitch ke ...,0
8,Since I started working immediately nor disfru...,1
9,My dad myself regano cuzs I was telling that t...,1


Classes are imbalanced as visible

## 2: Training/Validation Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.category.values, 
                                                  test_size=0.15, 
                                                  random_state=42,
                                                  stratify=df.category.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

In [ ]:
df.head()

,text,category,data_type
id,,,
1,Tonight peda segura,0,not_set
2,Eres such mala little woman shrew interesada,1,not_set
3,Yo kiero Pretzels lol,0,not_set
4,Fuck that nor ke the myself vaya in order to u...,1,not_set
5,I always tell my dad ke myself kiero kasar con...,1,not_set


In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['category', 'data_type']).count()

text
category data_type      
0        train      4770
         val         846
1        train      2601
         val         459
2        train      6981
         val        1234

# 3. Loading Tokenizer and Encoding our Data

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-cased',
    do_lower_case=True
)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
df.dropna()

,text,category,data_type
id,,,
1,Tonight peda segura,0,train
2,Eres such mala little woman shrew interesada,1,train
3,Yo kiero Pretzels lol,0,train
4,Fuck that nor ke the myself vaya in order to u...,1,train
5,I always tell my dad ke myself kiero kasar con...,1,train
...,...,...,...
16994,"lol allbaad nor tee dejas view lol , you only ...",2,train
16995,pound on in order to cold 40 and they tee quit...,0,train
16996,hahaa Fuhk you to tee doy somewhat pa that jal...,2,val


In [ ]:
df['text'] = df['text'].astype('str') 
df[df.data_type=='train'].text.values

array(['Tonight peda segura',
       'Eres such mala little woman shrew interesada',
       'Yo kiero Pretzels lol', ...,
       'pound on in order to cold 40 and they tee quita lol',
       "me pierdo peso and nor I wish quiera am off dieta lmao , I 've been getting tone lol I love waking up sore",
       'Please pay attention . Can I get your attention please ? 1 presta attention . 2 . Puedo have thy attention :)'],
      dtype=object)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].category.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].category.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
len(input_ids_train), len(attention_masks_train), len(labels_train)

(14444, 14444, 14444)

In [ ]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

In [ ]:
len(dataset_train)

14444

In [ ]:
dataset_val.tensors

(tensor([[  101,  1995,  2758,  ...,     0,     0,     0],
         [  101,  5871,   190,  ...,     0,     0,     0],
         [  101,  5871, 11631,  ...,     0,     0,     0],
         ...,
         [  101,   170,  1139,  ...,     0,     0,     0],
         [  101,  1137,  1991,  ...,     0,     0,     0],
         [  101,  5871,  2328,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([0, 2, 1,  ..., 2, 2, 2]))

# 4. Setting up BERT Pretrained Model

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-cased', 
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

# 5. Creating Data Loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

# 6. Setting Up Optimizer and Scheduler

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [ ]:
epochs = 1

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

# 7. Defining our Performance Metrics

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

# 8. Creating our Training Loop

In [ ]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/3611 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.9569803123921775


  0%|          | 0/80 [00:00<?, ?it/s]

Validation loss: 0.9336604326963425
F1 Score (weighted): 0.5228265632250735


# 9. Evaluating our Model

In [ ]:
accuracy_per_class(predictions, true_vals)


Class: neutral
Accuracy:253/851

Class: negative
Accuracy:197/459

Class: positive
Accuracy:931/1240



In [ ]:
print("Negative", (947/1240) * 100)
print("Positive", (530/755) * 100)
print("Neutral", (467/846) * 100)